In [56]:
import pandas as pd
import re
import json

In [57]:
INPUT_DATASET_PATH="data/input_dataset_v.1.xlsx"
UNITS_OF_MEASUREMENT_DATASET_PATH="data/units_of_measurement.csv"
CONVERSION_RATES_PATH="data/conversion_rates.csv"
UNITS_OF_MEASUREMENT_VARIATIONS_PATH="data/units_of_measurement_variations.json"

In [58]:
cash_flow_dataset = pd.read_excel(INPUT_DATASET_PATH)
cash_flow_dataset = cash_flow_dataset.map(lambda s: s.lower() if type(s) == str else s)

units_of_measurment_dataset = pd.read_csv(UNITS_OF_MEASUREMENT_DATASET_PATH)
units_of_measurment_dataset = units_of_measurment_dataset.map(lambda s: s.lower() if type(s) == str else s)

In [59]:
with open(UNITS_OF_MEASUREMENT_VARIATIONS_PATH) as json_reader:
    unit_variations = json.load(json_reader)

def standardize_units(text):
    for variation, standard_unit in unit_variations.items():
        if variation in text:
            text = text.replace(variation,standard_unit)

    return text

cash_flow_dataset['Description'] = cash_flow_dataset['Description'].apply(standardize_units)

In [60]:
all_units='|'.join(list(units_of_measurment_dataset['Unit']))
pattern = rf'([\d.,]+)\s*({all_units})\b'

cash_flow_dataset[['Amount', 'Unit']] = cash_flow_dataset['Description'].str.extract(pattern, flags=re.IGNORECASE)

In [61]:
def parse_amount_field(dataset):
    #LOGIC GOES HERE TO PARSE EUROPEAN AND INTERNATIONAL NUMBER FORMAT
    dataset['Amount'] = pd.to_numeric(dataset['Amount'], errors='coerce')
    
    return dataset

In [62]:
def standardize_units_of_measurement(dataset):
    df=pd.read_csv(CONVERSION_RATES_PATH)
    result = pd.merge(dataset,df, how='left')

    result['Unit'] = result['TargetUnit']
    result['Amount'] = result['Amount'] * result['ConversionRate']

    result = result.drop(['TargetUnit', 'ConversionRate'], axis=1)

    return result

cash_flow_dataset = parse_amount_field(cash_flow_dataset)
df = standardize_units_of_measurement(cash_flow_dataset)

df

,Transaction_ID,Date,Description,Vendor,GL_Account (General Ledger),Amount_EUR,IDEAL_RESULT,Amount,Unit
0,1001,2025-01-15,"invoice #45-a, agricultural diesel",agri-fuel inc.,raw materials expense,"1,200.00",agricultural diesel,NaN,NaN
1,1002,2025-01-20,electricity bill - offices,global power ltd.,utilities expense,350.00,electricity bill,NaN,NaN
2,1003,2025-01-22,payment for legal services,rossi legal firm,professional services,"2,500.00",payment for legal services,NaN,NaN
3,1004,2025-01-28,purchase ammonium nitrate 10t,fertilizers co.,raw materials expense,"7,000.00",ammonium nitrate,10000.0,kg
4,1005,2025-02-02,electricity bill - irrigation pumps,global power ltd.,utilities expense,"1,800.00",electricity bill,NaN,NaN
5,1006,2025-02-10,tractor t-100 maintenance,verdi mechanical shop,repairs & maintenance,600.00,tractor t-100 maintenance,NaN,NaN
6,1007,2025-02-15,"invoice #88-b, diesel fuel",agri-fuel inc.,raw materials expense,"1,500.00",diesel fuel,NaN,NaN
7,1008,2025-02-20,office stationery purchase,staples,general & admin expense,150.00,office stationery purchase,NaN,NaN
8,1009,2025-03-05,herbicide purchase – 500l,agrichem solutions,raw materials expense,3200,herbicide,500.0,l
9,1010,2025-03-08,water bill - irrigation system,municipal water authority,utilities expense,890,water bill,NaN,NaN
